## std::remove_if vs std::erase_if

Both take a range and a predicate. **std::remove_if** moves the elements to be deleted to the end of the container, and returns a new iterator to the new end. **std::erase_if** simplifies this process, erasing the elements from the container completely.

In [4]:
#include <iostream>
#include <vector>
#include <algorithm>

std::vector<int> numbers = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10};

auto is_even = [](int num){return num % 2 == 0;};
auto is_odd = [](int num){return num % 2 != 0;};

auto print_container = [](const auto& container){
                            for (const auto& elem : container)
                                std::cout << elem << " ";
                            std::cout << std::endl;
                        };

// Removes elements from the range [first, last) for which the predicate returns true.
auto new_end = std::remove_if(numbers.begin(), numbers.end(), is_even);

// To actually erase the elements, you need to use erase
numbers.erase(new_end, numbers.end());

print_container(numbers);


1 3 5 7 9 


In [ ]:
std::erase_if(numbers.begin(), numbers.end(), is_odd);

print_container(numbers);

input_line_18:6:1: error: type '(lambda at input_line_17:5:24)' does not provide a call operator
print_container(numbers);
^~~~~~~~~~~~~~~


## minHeap and maxHeap with std::priority_queue

In [1]:
#include <iostream>
#include <queue>

std::priority_queue<int, std::vector<int>, std::greater<int>> minHeap;
for (int i = 1; i <= 10; ++i)
    minHeap.push(i);

std::cout << minHeap.top();


1

In [3]:
std::priority_queue<int, std::vector<int>> maxHeap;
for (int i = 1; i <= 10; ++i)
    maxHeap.push(i);

std::cout << maxHeap.top();

10

## Structured binding

In [2]:
#include <iostream>
#include <unordered_map>

std::unordered_map<std::string, int> name_to_score;
name_to_score["A"] = 1;
name_to_score["B"] = 2;
name_to_score["C"] = 3;

for (auto [key, value] : name_to_score)
{
    std::cout << key << " : " << value << std::endl;
}


B : 2
C : 3
A : 1


## Custom hash function for use in std::unordered_set

In [1]:
#include <iostream>
#include <unordered_set>
#include <utility>

// Custom hash function for std::pair<int, int>
struct PairHash {
    template <class T1, class T2>
    size_t operator()(const std::pair<T1, T2>& p) const {
        // Combine the hashes of the two elements in the pair
        size_t hash1 = std::hash<T1>{}(p.first);
        size_t hash2 = std::hash<T2>{}(p.second);
        return hash1 ^ hash2;
    }
};

std::unordered_set<std::pair<int, int>, PairHash> set;
set.insert({1, 2});

## Speeding up input

**ios_base::sync_with_stdio(false)** disables the synchronization between C and C++ standard streams. Improves perforance as long as C and C++ I/O are not mixed.

**cin.tie(NULL)** unties cin from cout. When two streams are tied, one stream is flushed automatically before each I/O operation on the other stream.

In [ ]:
ios_base::sync_with_stdio(false);
cin.tie(NULL);

## Minimal implementation of unique_ptr

* **noexcept** is necessary for move constructor and move assignment operator in order to take advantage of certain optimizations.

* copy constructor and copy assignment are explicitly deleted.

In [ ]:
template <typename T>
class unique_ptr
{
public:
    explicit unique_ptr(T* ptr = nullptr)
    {
        ptr_ = ptr;
        ptr = nullptr;
    }

    unique_ptr(const unique_ptr& other) = delete;
    unique_ptr& operator=(const unique_ptr& other) = delete;

    unique_ptr(unique_ptr&& other) noexcept
    {
        ptr_ = other.ptr_;
        other.ptr_ = nullptr;
    }

    unique_ptr& operator=(unique_ptr&& other) noexcept
    {
        delete ptr_;

        ptr_ = other.ptr_;
        other.ptr_ = nullptr;

        return *this;
    }

    ~unique_ptr()
    {
        delete ptr_;
    }

    T* get()
    {
        return ptr_;
    }

private:
    T* ptr_;
};

## Fast parsing of large input files

When reading large files, it is best to do it in large chunks, in order to minimize the number of system calls, and to optimize disk access. A disk block size is typically 4096 bytes, which is the optimal size to read at once.

In [ ]:
class InParser{
private:
    const static size_t BLOCK_SIZE = 4096;

    FILE *fin;
    char *buffer;
    size_t current_index;

    inline char read_char()
    {
        ++current_index;
        if (current_index == BLOCK_SIZE)
        {
            current_index = 0;
            fread(buffer, 1, BLOCK_SIZE, fin);
        }
        return buffer[current_index];
    }

public:
    explicit InParser(const char* file_name)
    {
        fin = fopen(file_name, "r");
        if (fin == NULL)
            printf("File not found!");

        buffer = new char[BLOCK_SIZE]();
        current_index = BLOCK_SIZE - 1;
    }

    ~InParser()
    {
        delete[] buffer;
    }

    InParser& operator >> (int &num)
    {
        char ch;
        while (!isdigit(ch = read_char()) && ch != '-');
        int sign = 1;
        if (ch == '-')
        {
            num = 0;
            sign = -1;
        } else
        {
            num = ch - '0';
        }
        while (isdigit(ch = read_char()))
        {
            num = 10 * num + ch - '0';
        }
        num *= sign;
        return *this;
    }

    InParser& operator >> (long long &num)
    {
        char ch;
        num = 0;
        while (!isdigit(ch = read_char()) && ch != '-');
        long long sign = 1;
        if (ch == '-')
        {
            num = 0;
            sign = -1;
        } else
        {
            num = ch - '0';
        }
        while (isdigit(ch = read_char()))
        {
            num = 10 * num + ch - '0';
        }
        num *= sign;
        return *this;
    }
};